# 05 compare the different FDG runs / reference regions

versions needed to compare:

Data versions:
original reference region + original subject set
Paracentral-gm reference region + original subject set
Paracentral-gm reference region + new subject set
RAS-ref1 reference region + original subject set
RAS-ref1 reference region + new subject set



age correction of original data is for plasma data only, linear regression using all_data[all_data['source'].isin(['Control']) & all_data['Subject_Label'].isin(['AANDS', 'ADRC'])])

In [1]:
# install required packages - commented out so it doesn't install every time
#%conda install -n Lauren openpyxl numpy pandas statsmodels plotnine matplotlib scikit-learn scipy mizani nbconvert pandoc pyreadstat kmodes seaborn

# import required packages
import openpyxl; 
import datetime;
import numpy as np;
import pandas as pd;
import statsmodels as sm;
import statsmodels.formula.api as smf;
import plotnine as p9;
import matplotlib;
#import sklearn;
import scipy;
from scipy.stats import linregress;
from scipy.stats import pearsonr;
import itertools
from sklearn import preprocessing 
import inspect
import pickle


# read in and clean data

In [2]:
data_folder_loc = "//admsyn/Primary/ADM/CustomerStudies/Rockefeller/Riluzole_Biomarkers/"
code_folder_loc = "//admsyn/homes/@DH-ADMDX/0/lauren.koenig-1606/code/Riluzole FDG/"

In [3]:
# read in data
results_composite, temp, data1 = pd.read_pickle(code_folder_loc + '/output/04_data_composite_rr.pkl')
results_pons, temp, data2 = pd.read_pickle(code_folder_loc + '/output/04_data_pons_rr.pkl')
results_para, temp, data3 = pd.read_pickle(code_folder_loc + '/output/04_data_para_rr.pkl')
results_new_para, temp, data4 = pd.read_pickle(code_folder_loc + '/output/04_data_new_para_rr.pkl')
results_ras, temp, data5 = pd.read_pickle(code_folder_loc + '/output/04_data_ras_rr.pkl')
results_subset_ras, temp, data5_subset = pd.read_pickle(code_folder_loc + '/output/04_data_subset_ras_rr.pkl')




In [4]:
rename_columns = ['cor_p_baseline', 'cor_r_baseline', 'cor_n',
       'mv_age_p_baseline', 'mv_age_B_baseline', 'mv_age_n',
       'mv_ageedu_p_baseline', 'mv_ageedi_B_baseline', 'mv_ageedu_n',
       'mv_aeasr_p_baseline', 'mv_aeasr_B_baseline', 'mv_aeasr_n']

In [5]:
results_composite = results_composite.rename(columns = dict(zip(rename_columns, 
['composite_' + ele for ele in rename_columns])))

results_pons = results_pons.rename(columns = dict(zip(rename_columns, 
['pons_' + ele for ele in rename_columns])))

results_para = results_para.rename(columns = dict(zip(rename_columns, 
['para_' + ele for ele in rename_columns])))

results_new_para = results_new_para.rename(columns = dict(zip(rename_columns, 
['new_para_' + ele for ele in rename_columns])))

results_ras = results_ras.rename(columns = dict(zip(rename_columns, 
['ras_' + ele for ele in rename_columns])))

results_subset_ras = results_subset_ras.rename(columns = dict(zip(rename_columns, 
['subset_ras_' + ele for ele in rename_columns])))


In [6]:
# define biomarker groups
FDG_columns = ['Avg_MedOrbFrontal', 'Graycer_gm', 'Avg_PCC', 'Avg_Hip', 'MTL_gm', 'CO', 'Temp', 'SensMot_gm', 'L_Hip',
 'Vermis_gm', 'FRONTAL_gm', 'AC_gm', 'Precun_gm', 'Par_gm', 'RASref1_gm', 'Temp_gm', 'Paracentral_gm', 'R_Hip', 'PostCing_gm']

plasma_columns = [ 'Ab40', 'Ab42', 'Ab42_40', 'GFAP','NFL',  'pTau181', 'pTau217',  'pTau231', 'pTau181_Ab42']
log10_plasma_columns = [s + '_log10' for s in plasma_columns]

cog_columns = ['MMSE', 'adascogtotal', 'bvrt', 'dstotal','tma','tmb',  'cowattotal', 'cdrtotal', 'cdrsum', 'adltotal', 'npitotal', 'gds']
vol_columns = ['VOL_Precun_Lz', 'VOL_Precun_Rz', 'VOL_InfPar_Lz', 'VOL_InfPar_Rz', 'VOL_Hip_Lz', 'VOL_Hip_Rz', 'VOL_TotalGrayz', 'VOL_Inf_Mid_Fus_Temp_Lz', 
'VOL_Inf_Mid_Fus_Temp_Rz','VOL__MidFront_Lz', 'VOL_MidFront_Rz']



# Composite vs New Para

In [7]:
comp_new_para = pd.merge(results_composite, results_new_para, how = 'inner')
comp_new_para

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
0,Ab40,MTL_gm,0.231527,-0.204507,36,0.859912,-0.033357,36,0.765629,-0.065592,...,41,0.879090,-0.025949,40,0.764854,-0.069336,36,0.998305,0.000561,35
1,Ab40,FRONTAL_gm,0.779819,-0.048269,36,0.533655,0.108518,36,0.556243,0.110970,...,41,0.435330,0.128753,40,0.556341,0.114583,36,0.529761,0.140518,35
2,Ab40,AC_gm,0.796487,-0.044533,36,0.926577,0.016909,36,0.965413,0.008416,...,41,0.826350,-0.037865,40,0.964003,0.008925,36,0.941134,0.017565,35
3,Ab40,Precun_gm,0.982164,-0.003862,36,0.991163,0.001944,36,0.962639,-0.008797,...,41,0.857469,0.030100,40,0.959309,-0.009863,36,0.899370,-0.028402,35
4,Ab40,Par_gm,0.922244,0.016862,36,0.704620,0.068971,36,0.731849,0.064970,...,41,0.766364,0.051274,40,0.731164,0.064462,36,0.734919,0.070023,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,pTau181_Ab42,VOL_TotalGrayz,0.173986,0.280769,25,0.083519,0.372222,25,0.016493,0.510871,...,25,0.083519,0.372222,25,0.016493,0.510871,25,0.014410,0.635560,24
338,pTau181_Ab42,VOL_Inf_Mid_Fus_Temp_Lz,0.234069,0.246923,25,0.073877,0.379132,25,0.004400,0.612460,...,25,0.073877,0.379132,25,0.004400,0.612460,25,0.036048,0.565068,24
339,pTau181_Ab42,VOL_Inf_Mid_Fus_Temp_Rz,0.195769,0.267692,25,0.099014,0.343541,25,0.006681,0.578515,...,25,0.099014,0.343541,25,0.006681,0.578515,25,0.063452,0.569734,24
340,pTau181_Ab42,VOL__MidFront_Lz,0.742287,0.069231,25,0.660397,0.110217,25,0.614977,0.123332,...,25,0.660397,0.110217,25,0.614977,0.123332,25,0.462537,0.204695,24


## cor

In [8]:
comp_new_para[(comp_new_para['composite_cor_p_baseline'] < 0.05) & (comp_new_para['new_para_cor_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
17,Ab42_40,Precun_gm,0.043701,0.358852,32,0.020231,0.411517,32,0.020959,0.433611,...,37,0.02459,0.372345,36,0.020774,0.443357,32,0.008954,0.528315,31


In [9]:
comp_new_para[(comp_new_para['composite_cor_p_baseline'] >= 0.05) & (comp_new_para['new_para_cor_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
10,Ab42,Precun_gm,0.092348,0.302558,32,0.107874,0.293635,32,0.163857,0.267537,...,37,0.067379,0.305760,36,0.164171,0.265191,32,0.388394,0.196199,31
19,Ab42_40,Temp_gm,0.085760,0.308553,32,0.147717,0.262477,32,0.158247,0.271622,...,37,0.107181,0.266753,36,0.158477,0.297319,32,0.101392,0.358837,31


In [10]:
(comp_new_para['composite_cor_p_baseline']  - comp_new_para['new_para_cor_p_baseline']).describe()

count    342.000000
mean       0.001758
std        0.064049
min       -0.354909
25%        0.000000
50%        0.000000
75%        0.000000
max        0.341366
dtype: float64

## age

In [12]:
comp_new_para.loc[(comp_new_para['composite_mv_age_p_baseline'] < 0.05) & (comp_new_para['new_para_mv_age_p_baseline'] >= 0.05), ['var1', 'var2', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n', 'new_para_mv_age_p_baseline', 'new_para_mv_age_B_baseline',
       'new_para_mv_age_n']]

,var1,var2,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n
43,pTau217,FRONTAL_gm,0.043308,-0.350427,35,0.050892,-0.317208,38


In [13]:
comp_new_para[(comp_new_para['composite_mv_age_p_baseline'] >= 0.05) & (comp_new_para['new_para_mv_age_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [14]:
(comp_new_para['composite_mv_age_p_baseline']  - comp_new_para['new_para_mv_age_p_baseline'])[(comp_new_para['composite_mv_age_p_baseline']  - comp_new_para['new_para_mv_age_p_baseline']) != 0].describe().round(2)

count    98.00
mean      0.00
std       0.09
min      -0.30
25%      -0.01
50%      -0.00
75%       0.02
max       0.35
dtype: float64

In [15]:
(comp_new_para['composite_mv_age_p_baseline']  - comp_new_para['new_para_mv_age_p_baseline']).describe()

count    342.000000
mean       0.000442
std        0.048588
min       -0.298569
25%        0.000000
50%        0.000000
75%        0.000000
max        0.347573
dtype: float64

In [16]:
(comp_new_para['composite_mv_age_n']  - comp_new_para['new_para_mv_age_n']).value_counts()

 0    279
-4     56
-3      7
dtype: int64

## age education

In [17]:
comp_new_para[(comp_new_para['composite_mv_ageedu_p_baseline'] < 0.05) & (comp_new_para['new_para_mv_ageedu_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [18]:
comp_new_para[(comp_new_para['composite_mv_ageedu_p_baseline'] >= 0.05) & (comp_new_para['new_para_mv_ageedu_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [19]:
(comp_new_para['composite_mv_ageedu_p_baseline']  - comp_new_para['new_para_mv_ageedu_p_baseline']).describe()

count    342.000000
mean       0.000012
std        0.000790
min       -0.004577
25%        0.000000
50%        0.000000
75%        0.000000
max        0.005143
dtype: float64

## age education apoe sex race

In [20]:
comp_new_para.columns

Index(['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline',
       'composite_cor_n', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n',
       'composite_mv_ageedu_p_baseline', 'composite_mv_ageedi_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type',
       'new_para_cor_p_baseline', 'new_para_cor_r_baseline', 'new_para_cor_n',
       'new_para_mv_age_p_baseline', 'new_para_mv_age_B_baseline',
       'new_para_mv_age_n', 'new_para_mv_ageedu_p_baseline',
       'new_para_mv_ageedi_B_baseline', 'new_para_mv_ageedu_n',
       'new_para_mv_aeasr_p_baseline', 'new_para_mv_aeasr_B_baseline',
       'new_para_mv_aeasr_n'],
      dtype='object')

In [21]:
comp_new_para[(comp_new_para['composite_mv_aeasr_p_baseline'] < 0.05) & (comp_new_para['new_para_mv_aeasr_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [22]:
comp_new_para[(comp_new_para['composite_mv_aeasr_p_baseline'] >= 0.05) & (comp_new_para['new_para_mv_aeasr_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [23]:
(comp_new_para['composite_mv_aeasr_p_baseline']  - comp_new_para['new_para_mv_aeasr_p_baseline']).describe()

count    342.000000
mean      -0.000025
std        0.000874
min       -0.007378
25%        0.000000
50%        0.000000
75%        0.000000
max        0.003082
dtype: float64

# Ras vs subset RAS

In [24]:
ras_ras = pd.merge(results_ras, results_subset_ras, how = 'inner')
ras_ras

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,subset_ras_cor_n,subset_ras_mv_age_p_baseline,subset_ras_mv_age_B_baseline,subset_ras_mv_age_n,subset_ras_mv_ageedu_p_baseline,subset_ras_mv_ageedi_B_baseline,subset_ras_mv_ageedu_n,subset_ras_mv_aeasr_p_baseline,subset_ras_mv_aeasr_B_baseline,subset_ras_mv_aeasr_n
0,Ab40,Graycer_gm,0.732740,0.058945,36,0.876948,0.027032,36,0.890963,0.024400,...,36,0.876948,0.027032,36,0.890963,0.024400,36,0.745108,0.064010,35
1,Ab40,Avg_PCC,0.714821,0.063063,36,0.634187,0.085258,36,0.650798,0.082811,...,36,0.634187,0.085258,36,0.650798,0.082811,36,0.840136,0.041739,35
2,Ab40,MTL_gm,0.220662,-0.195470,41,0.479591,-0.119138,40,0.802658,-0.051721,...,36,0.827926,-0.039159,36,0.802658,-0.046467,36,0.956688,-0.011012,35
3,Ab40,CO,0.632949,0.082368,36,0.216581,0.216459,36,0.224126,0.216207,...,36,0.216581,0.216459,36,0.224126,0.216207,36,0.217937,0.251309,35
4,Ab40,SensMot_gm,0.137310,0.236063,41,0.719092,0.062469,40,0.091573,0.417331,...,36,0.118714,0.279402,36,0.091573,0.322298,36,0.096942,0.334124,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,pTau181_Ab42,VOL_TotalGrayz,0.173986,0.280769,25,0.083519,0.372222,25,0.016493,0.510871,...,25,0.083519,0.372222,25,0.016493,0.510871,25,0.014410,0.635560,24
401,pTau181_Ab42,VOL_Inf_Mid_Fus_Temp_Lz,0.234069,0.246923,25,0.073877,0.379132,25,0.004400,0.612460,...,25,0.073877,0.379132,25,0.004400,0.612460,25,0.036048,0.565068,24
402,pTau181_Ab42,VOL_Inf_Mid_Fus_Temp_Rz,0.195769,0.267692,25,0.099014,0.343541,25,0.006681,0.578515,...,25,0.099014,0.343541,25,0.006681,0.578515,25,0.063452,0.569734,24
403,pTau181_Ab42,VOL__MidFront_Lz,0.742287,0.069231,25,0.660397,0.110217,25,0.614977,0.123332,...,25,0.660397,0.110217,25,0.614977,0.123332,25,0.462537,0.204695,24


## cor

In [25]:
ras_ras[(ras_ras['ras_cor_p_baseline'] < 0.05) & (ras_ras['subset_ras_cor_p_baseline'] >= 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,subset_ras_cor_n,subset_ras_mv_age_p_baseline,subset_ras_mv_age_B_baseline,subset_ras_mv_age_n,subset_ras_mv_ageedu_p_baseline,subset_ras_mv_ageedi_B_baseline,subset_ras_mv_ageedu_n,subset_ras_mv_aeasr_p_baseline,subset_ras_mv_aeasr_B_baseline,subset_ras_mv_aeasr_n
36,Ab42_40,Precun_gm,0.018150,0.386439,37,0.008529,0.455392,36,0.014838,0.495647,...,32,0.012776,0.454670,32,0.014838,0.453439,32,0.004641,0.523910,31
39,Ab42_40,Temp_gm,0.027138,0.363205,37,0.070157,0.310858,36,0.136081,0.327816,...,32,0.126557,0.275610,32,0.136081,0.273921,32,0.074755,0.325326,31
95,pTau217,Temp_gm,0.035097,-0.338412,39,0.105756,-0.276941,38,0.061679,-0.389363,...,35,0.058282,-0.329032,35,0.061679,-0.330054,35,0.035015,-0.351897,34
130,Ab40,pTau181,0.041350,-0.312467,43,0.117783,-0.259754,40,0.193224,-0.242634,...,38,0.182368,-0.232060,36,0.193224,-0.231929,36,0.301894,-0.212976,35
140,Ab42,pTau231,0.041066,-0.337502,37,0.075697,-0.304300,34,0.169027,-0.255480,...,32,0.140850,-0.281152,30,0.169027,-0.261022,30,0.346202,-0.216587,29
154,GFAP,pTau181,0.028186,0.327305,45,0.045342,0.314916,42,0.076072,0.295880,...,40,0.060970,0.303634,38,0.076072,0.296256,38,0.266473,0.199684,37
166,pTau181,Ab40,0.041350,-0.312467,43,0.117783,-0.249549,40,0.193224,-0.215605,...,38,0.182368,-0.229401,36,0.193224,-0.225557,36,0.301894,-0.185053,35
169,pTau181,GFAP,0.028186,0.327305,45,0.045342,0.313735,42,0.076072,0.302963,...,40,0.060970,0.318591,38,0.076072,0.302578,38,0.266473,0.212299,37
183,pTau231,Ab42,0.041066,-0.337502,37,0.075697,-0.322855,34,0.169027,-0.279758,...,32,0.140850,-0.279399,30,0.169027,-0.273818,30,0.346202,-0.195511,29


In [26]:
ras_ras[(ras_ras['ras_cor_p_baseline'] >= 0.05) & (ras_ras['subset_ras_cor_p_baseline'] < 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,subset_ras_cor_n,subset_ras_mv_age_p_baseline,subset_ras_mv_age_B_baseline,subset_ras_mv_age_n,subset_ras_mv_ageedu_p_baseline,subset_ras_mv_ageedi_B_baseline,subset_ras_mv_ageedu_n,subset_ras_mv_aeasr_p_baseline,subset_ras_mv_aeasr_B_baseline,subset_ras_mv_aeasr_n
4,Ab40,SensMot_gm,0.13731,0.236063,41,0.719092,0.062469,40,0.091573,0.417331,...,36,0.118714,0.279402,36,0.091573,0.322298,36,0.096942,0.334124,35


In [27]:
(ras_ras['ras_cor_p_baseline']  - ras_ras['subset_ras_cor_p_baseline']).describe()

count    396.000000
mean      -0.001730
std        0.122529
min       -0.707240
25%        0.000000
50%        0.000000
75%        0.000000
max        0.668037
dtype: float64

## age

In [28]:
ras_ras[(ras_ras['ras_mv_age_p_baseline'] < 0.05) & (ras_ras['subset_ras_mv_age_p_baseline'] >= 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,subset_ras_cor_n,subset_ras_mv_age_p_baseline,subset_ras_mv_age_B_baseline,subset_ras_mv_age_n,subset_ras_mv_ageedu_p_baseline,subset_ras_mv_ageedi_B_baseline,subset_ras_mv_ageedu_n,subset_ras_mv_aeasr_p_baseline,subset_ras_mv_aeasr_B_baseline,subset_ras_mv_aeasr_n
54,GFAP,Paracentral_gm,0.031435,0.328614,43,0.035655,0.344853,42,0.116686,0.336140,...,38,0.091437,0.281722,38,0.116686,0.290585,38,0.129450,0.280605,37
100,pTau231,MTL_gm,0.031618,0.344737,39,0.043594,0.340758,38,0.086139,0.358128,...,34,0.081522,0.316480,34,0.086139,0.319098,34,0.218434,0.246988,33
114,pTau181_Ab42,MTL_gm,0.109239,0.267662,37,0.039897,0.341526,36,0.103240,0.317536,...,32,0.146989,0.272117,32,0.103240,0.304507,32,0.162043,0.270527,31
154,GFAP,pTau181,0.028186,0.327305,45,0.045342,0.314916,42,0.076072,0.295880,...,40,0.060970,0.303634,38,0.076072,0.296256,38,0.266473,0.199684,37
169,pTau181,GFAP,0.028186,0.327305,45,0.045342,0.313735,42,0.076072,0.302963,...,40,0.060970,0.318591,38,0.076072,0.302578,38,0.266473,0.212299,37


In [29]:
ras_ras[(ras_ras['ras_mv_age_p_baseline'] >= 0.05) & (ras_ras['subset_ras_mv_age_p_baseline'] < 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,subset_ras_cor_n,subset_ras_mv_age_p_baseline,subset_ras_mv_age_B_baseline,subset_ras_mv_age_n,subset_ras_mv_ageedu_p_baseline,subset_ras_mv_ageedi_B_baseline,subset_ras_mv_ageedu_n,subset_ras_mv_aeasr_p_baseline,subset_ras_mv_aeasr_B_baseline,subset_ras_mv_aeasr_n
164,NFL,pTau231,0.147999,0.23296,40,0.07647,0.297715,37,0.042804,0.368909,...,35,0.037017,0.362746,33,0.042804,0.363551,33,0.069114,0.378537,32
186,pTau231,NFL,0.147999,0.23296,40,0.07647,0.300318,37,0.042804,0.363342,...,35,0.037017,0.377801,33,0.042804,0.368697,33,0.069114,0.346331,32


In [30]:
(ras_ras['ras_mv_age_p_baseline']  - ras_ras['subset_ras_mv_age_p_baseline']).describe()

count    3.960000e+02
mean    -1.594287e-02
std      1.408462e-01
min     -6.259493e-01
25%     -1.929550e-10
50%      0.000000e+00
75%      0.000000e+00
max      6.653127e-01
dtype: float64

## age education

In [31]:
ras_ras[(ras_ras['ras_mv_ageedu_p_baseline'] < 0.05) & (ras_ras['subset_ras_mv_ageedu_p_baseline'] >= 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,subset_ras_cor_n,subset_ras_mv_age_p_baseline,subset_ras_mv_age_B_baseline,subset_ras_mv_age_n,subset_ras_mv_ageedu_p_baseline,subset_ras_mv_ageedi_B_baseline,subset_ras_mv_ageedu_n,subset_ras_mv_aeasr_p_baseline,subset_ras_mv_aeasr_B_baseline,subset_ras_mv_aeasr_n


In [32]:
ras_ras[(ras_ras['ras_mv_ageedu_p_baseline'] >= 0.05) & (ras_ras['subset_ras_mv_ageedu_p_baseline'] < 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,subset_ras_cor_n,subset_ras_mv_age_p_baseline,subset_ras_mv_age_B_baseline,subset_ras_mv_age_n,subset_ras_mv_ageedu_p_baseline,subset_ras_mv_ageedi_B_baseline,subset_ras_mv_ageedu_n,subset_ras_mv_aeasr_p_baseline,subset_ras_mv_aeasr_B_baseline,subset_ras_mv_aeasr_n


In [33]:
(ras_ras['ras_mv_ageedu_p_baseline']  - ras_ras['subset_ras_mv_ageedu_p_baseline']).describe()

count    3.960000e+02
mean    -2.116220e-17
std      2.588727e-16
min     -2.997602e-15
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      6.661338e-16
dtype: float64

## age education apoe sex race

In [34]:
ras_ras[(ras_ras['ras_mv_aeasr_p_baseline'] < 0.05) & (ras_ras['subset_ras_mv_aeasr_p_baseline'] >= 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,subset_ras_cor_n,subset_ras_mv_age_p_baseline,subset_ras_mv_age_B_baseline,subset_ras_mv_age_n,subset_ras_mv_ageedu_p_baseline,subset_ras_mv_ageedi_B_baseline,subset_ras_mv_ageedu_n,subset_ras_mv_aeasr_p_baseline,subset_ras_mv_aeasr_B_baseline,subset_ras_mv_aeasr_n


In [35]:
ras_ras[(ras_ras['ras_mv_aeasr_p_baseline'] >= 0.05) & (ras_ras['subset_ras_mv_aeasr_p_baseline'] < 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,subset_ras_cor_n,subset_ras_mv_age_p_baseline,subset_ras_mv_age_B_baseline,subset_ras_mv_age_n,subset_ras_mv_ageedu_p_baseline,subset_ras_mv_ageedi_B_baseline,subset_ras_mv_ageedu_n,subset_ras_mv_aeasr_p_baseline,subset_ras_mv_aeasr_B_baseline,subset_ras_mv_aeasr_n


In [36]:
(ras_ras['ras_mv_aeasr_p_baseline']  - ras_ras['subset_ras_mv_aeasr_p_baseline']).describe()

count    3.960000e+02
mean    -7.486031e-17
std      1.154726e-15
min     -6.661338e-15
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.043610e-14
dtype: float64

# Ras vs new para

In [37]:
ras_new_para = pd.merge(results_ras, results_new_para, how = 'inner')
ras_new_para

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
0,Ab40,MTL_gm,0.220662,-0.195470,41,0.479591,-0.119138,40,0.802658,-0.051721,...,41,0.879090,-0.025949,40,0.764854,-0.069336,36,0.998305,0.000561,35
1,Ab40,SensMot_gm,0.137310,0.236063,41,0.719092,0.062469,40,0.091573,0.417331,...,41,0.063042,0.301542,40,0.031438,0.400301,36,0.025934,0.440487,35
2,Ab40,FRONTAL_gm,0.925925,0.014983,41,0.967216,-0.007045,40,0.644089,0.091602,...,41,0.435330,0.128753,40,0.556341,0.114583,36,0.529761,0.140518,35
3,Ab40,AC_gm,0.814467,-0.037805,41,0.505365,-0.112082,40,0.966904,0.008607,...,41,0.826350,-0.037865,40,0.964003,0.008925,36,0.941134,0.017565,35
4,Ab40,Precun_gm,0.774399,-0.046167,41,0.561828,-0.101348,40,0.896941,-0.026170,...,41,0.857469,0.030100,40,0.959309,-0.009863,36,0.899370,-0.028402,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,pTau181_Ab42,VOL_TotalGrayz,0.173986,0.280769,25,0.083519,0.372222,25,0.016493,0.510871,...,25,0.083519,0.372222,25,0.016493,0.510871,25,0.014410,0.635560,24
365,pTau181_Ab42,VOL_Inf_Mid_Fus_Temp_Lz,0.234069,0.246923,25,0.073877,0.379132,25,0.004400,0.612460,...,25,0.073877,0.379132,25,0.004400,0.612460,25,0.036048,0.565068,24
366,pTau181_Ab42,VOL_Inf_Mid_Fus_Temp_Rz,0.195769,0.267692,25,0.099014,0.343541,25,0.006681,0.578515,...,25,0.099014,0.343541,25,0.006681,0.578515,25,0.063452,0.569734,24
367,pTau181_Ab42,VOL__MidFront_Lz,0.742287,0.069231,25,0.660397,0.110217,25,0.614977,0.123332,...,25,0.660397,0.110217,25,0.614977,0.123332,25,0.462537,0.204695,24


## cor

In [38]:
ras_new_para[(ras_new_para['ras_cor_p_baseline'] < 0.05) & (ras_new_para['new_para_cor_p_baseline'] >= 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
24,Ab42_40,Precun_gm,0.01815,0.386439,37,0.008529,0.455392,36,0.014838,0.495647,...,37,0.02459,0.372345,36,0.020774,0.443357,32,0.008954,0.528315,31


In [39]:
ras_new_para[(ras_new_para['ras_cor_p_baseline'] >= 0.05) & (ras_new_para['new_para_cor_p_baseline'] < 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
14,Ab42,Precun_gm,0.141233,0.246577,37,0.293402,0.189348,36,0.130378,0.306524,...,37,0.067379,0.305760,36,0.164171,0.265191,32,0.388394,0.196199,31
25,Ab42_40,Par_gm,0.102556,0.272641,37,0.078800,0.309816,36,0.130197,0.325396,...,37,0.118173,0.262693,36,0.165034,0.258430,32,0.128537,0.290084,31
32,GFAP,FRONTAL_gm,0.648282,-0.071582,43,0.728047,-0.058122,42,0.458673,-0.134771,...,43,0.022479,-0.356626,42,0.041756,-0.349772,38,0.051380,-0.343251,37
61,pTau217,SensMot_gm,0.172766,-0.222807,39,0.862597,-0.030740,38,0.667615,-0.109505,...,39,0.077152,-0.291165,38,0.021570,-0.440246,35,0.024095,-0.422182,34
62,pTau217,FRONTAL_gm,0.175383,-0.221491,39,0.437936,-0.133965,38,0.372160,-0.179521,...,39,0.050892,-0.317208,38,0.033321,-0.384078,35,0.075504,-0.324276,34
64,pTau217,Precun_gm,0.081954,-0.282027,39,0.204307,-0.221131,38,0.191805,-0.261729,...,39,0.028535,-0.356517,38,0.027162,-0.397276,35,0.127457,-0.281929,34


In [40]:
(ras_new_para['ras_cor_p_baseline']  - ras_new_para['new_para_cor_p_baseline']).describe()

count    351.000000
mean       0.012199
std        0.123396
min       -0.528871
25%        0.000000
50%        0.000000
75%        0.000000
max        0.861908
dtype: float64

## age

In [41]:
ras_new_para.loc[(ras_new_para['ras_mv_age_p_baseline'] < 0.05) & (ras_new_para['new_para_mv_age_p_baseline'] >= 0.05), ['var1', 'var2',
'ras_mv_age_p_baseline', 'ras_mv_age_B_baseline', 'ras_mv_age_n',  'new_para_mv_age_p_baseline', 'new_para_mv_age_B_baseline',
       'new_para_mv_age_n']]

,var1,var2,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n
80,pTau181_Ab42,MTL_gm,0.039897,0.341526,36,0.20027,0.221264,36


In [42]:
ras_new_para.loc[(ras_new_para['ras_mv_age_p_baseline'] >= 0.05) & (ras_new_para['new_para_mv_age_p_baseline'] < 0.05), ['var1', 'var2',
'ras_mv_age_p_baseline', 'ras_mv_age_B_baseline', 'ras_mv_age_n',  'new_para_mv_age_p_baseline', 'new_para_mv_age_B_baseline',
       'new_para_mv_age_n']]

,var1,var2,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n
32,GFAP,FRONTAL_gm,0.728047,-0.058122,42,0.022479,-0.356626,42
34,GFAP,Precun_gm,0.534249,-0.105982,42,0.018262,-0.372406,42
35,GFAP,Par_gm,0.331243,-0.170836,42,0.031328,-0.351754,42
64,pTau217,Precun_gm,0.204307,-0.221131,38,0.028535,-0.356517,38
67,pTau217,Temp_gm,0.105756,-0.276941,38,0.008761,-0.417733,38


In [43]:
(ras_new_para['ras_mv_age_p_baseline']  - ras_new_para['new_para_mv_age_p_baseline']).describe()

count    351.000000
mean       0.007030
std        0.178022
min       -0.607923
25%        0.000000
50%        0.000000
75%        0.000000
max        0.785445
dtype: float64

## age education

In [44]:
ras_new_para[(ras_new_para['ras_mv_ageedu_p_baseline'] < 0.05) & (ras_new_para['new_para_mv_ageedu_p_baseline'] >= 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [45]:
ras_new_para[(ras_new_para['ras_mv_ageedu_p_baseline'] >= 0.05) & (ras_new_para['new_para_mv_ageedu_p_baseline'] < 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
1,Ab40,SensMot_gm,0.137310,0.236063,41,0.719092,0.062469,40,0.091573,0.417331,...,41,0.063042,0.301542,40,0.031438,0.400301,36,0.025934,0.440487,35
11,Ab42,SensMot_gm,0.382582,0.147828,37,0.872199,0.028808,36,0.098664,0.430668,...,37,0.091302,0.279539,36,0.044559,0.394320,32,0.082676,0.392566,31
32,GFAP,FRONTAL_gm,0.648282,-0.071582,43,0.728047,-0.058122,42,0.458673,-0.134771,...,43,0.022479,-0.356626,42,0.041756,-0.349772,38,0.051380,-0.343251,37
34,GFAP,Precun_gm,0.960906,-0.007702,43,0.534249,-0.105982,42,0.353819,-0.171248,...,43,0.018262,-0.372406,42,0.045915,-0.341150,38,0.115938,-0.284392,37
35,GFAP,Par_gm,0.644124,-0.072488,43,0.331243,-0.170836,42,0.108530,-0.306770,...,43,0.031328,-0.351754,42,0.021608,-0.379371,38,0.071454,-0.303397,37
61,pTau217,SensMot_gm,0.172766,-0.222807,39,0.862597,-0.030740,38,0.667615,-0.109505,...,39,0.077152,-0.291165,38,0.021570,-0.440246,35,0.024095,-0.422182,34
62,pTau217,FRONTAL_gm,0.175383,-0.221491,39,0.437936,-0.133965,38,0.372160,-0.179521,...,39,0.050892,-0.317208,38,0.033321,-0.384078,35,0.075504,-0.324276,34
64,pTau217,Precun_gm,0.081954,-0.282027,39,0.204307,-0.221131,38,0.191805,-0.261729,...,39,0.028535,-0.356517,38,0.027162,-0.397276,35,0.127457,-0.281929,34
67,pTau217,Temp_gm,0.035097,-0.338412,39,0.105756,-0.276941,38,0.061679,-0.389363,...,39,0.008761,-0.417733,38,0.004213,-0.515409,35,0.001588,-0.550945,34
70,pTau231,MTL_gm,0.031618,0.344737,39,0.043594,0.340758,38,0.086139,0.358128,...,39,0.042447,0.345532,38,0.020698,0.504003,34,0.071912,0.463527,33


In [46]:
(ras_new_para['ras_mv_ageedu_p_baseline']  - ras_new_para['new_para_mv_ageedu_p_baseline']).describe()

count    351.000000
mean       0.014755
std        0.102731
min       -0.356771
25%        0.000000
50%        0.000000
75%        0.000000
max        0.888425
dtype: float64

## age education apoe sex race

In [47]:
ras_new_para[(ras_new_para['ras_mv_aeasr_p_baseline'] < 0.05) & (ras_new_para['new_para_mv_aeasr_p_baseline'] >= 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [48]:
ras_new_para[(ras_new_para['ras_mv_aeasr_p_baseline'] >= 0.05) & (ras_new_para['new_para_mv_aeasr_p_baseline'] < 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedi_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
1,Ab40,SensMot_gm,0.137310,0.236063,41,0.719092,0.062469,40,0.091573,0.417331,...,41,0.063042,0.301542,40,0.031438,0.400301,36,0.025934,0.440487,35
61,pTau217,SensMot_gm,0.172766,-0.222807,39,0.862597,-0.030740,38,0.667615,-0.109505,...,39,0.077152,-0.291165,38,0.021570,-0.440246,35,0.024095,-0.422182,34


In [49]:
(ras_new_para['ras_mv_aeasr_p_baseline']  - ras_new_para['new_para_mv_aeasr_p_baseline']).describe()

count    351.000000
mean       0.010540
std        0.104502
min       -0.554418
25%        0.000000
50%        0.000000
75%        0.000000
max        0.874432
dtype: float64

# Composite vs Old Para

In [50]:
comp_para = pd.merge(results_composite, results_para, how = 'inner')
comp_para

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedi_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
0,Ab40,Avg_PCC,0.833555,0.036293,36,0.589517,0.094711,36,0.615900,0.095709,...,36,0.910374,0.020038,36,0.925266,0.017039,36,0.830950,-0.043948,35
1,Ab40,MTL_gm,0.231527,-0.204507,36,0.859912,-0.033357,36,0.765629,-0.065592,...,36,0.559269,-0.104352,36,0.545423,-0.110686,36,0.611618,-0.103299,35
2,Ab40,AC_gm,0.796487,-0.044533,36,0.926577,0.016909,36,0.965413,0.008416,...,36,0.730048,-0.061094,36,0.733944,-0.061095,36,0.683330,-0.080826,35
3,Ab40,Precun_gm,0.982164,-0.003862,36,0.991163,0.001944,36,0.962639,-0.008797,...,36,0.650847,-0.080079,36,0.651068,-0.081292,36,0.544580,-0.121715,35
4,Ab40,Par_gm,0.922244,0.016862,36,0.704620,0.068971,36,0.731849,0.064970,...,36,0.891571,0.025786,36,0.898092,0.024612,36,0.961421,0.010534,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,pTau181_Ab42,VOL_TotalGrayz,0.173986,0.280769,25,0.083519,0.372222,25,0.016493,0.510871,...,25,0.083519,0.372222,25,0.016493,0.510871,25,0.014410,0.635560,24
338,pTau181_Ab42,VOL_Inf_Mid_Fus_Temp_Lz,0.234069,0.246923,25,0.073877,0.379132,25,0.004400,0.612460,...,25,0.073877,0.379132,25,0.004400,0.612460,25,0.036048,0.565068,24
339,pTau181_Ab42,VOL_Inf_Mid_Fus_Temp_Rz,0.195769,0.267692,25,0.099014,0.343541,25,0.006681,0.578515,...,25,0.099014,0.343541,25,0.006681,0.578515,25,0.063452,0.569734,24
340,pTau181_Ab42,VOL__MidFront_Lz,0.742287,0.069231,25,0.660397,0.110217,25,0.614977,0.123332,...,25,0.660397,0.110217,25,0.614977,0.123332,25,0.462537,0.204695,24


## cor

In [51]:
comp_para[(comp_para['composite_cor_p_baseline'] < 0.05) & (comp_para['para_cor_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedi_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
7,Ab42,Avg_PCC,0.049509,0.350073,32,0.074420,0.325552,32,0.119948,0.302711,...,32,0.161924,0.266659,32,0.190225,0.252185,32,0.361139,0.207207,31
18,Ab42_40,Par_gm,0.032843,0.378150,32,0.150872,0.263692,32,0.164547,0.262936,...,32,0.037223,0.388456,32,0.040092,0.389185,32,0.034491,0.420310,31
36,pTau181,MTL_gm,0.020826,0.373700,38,0.026257,0.388571,38,0.002135,0.567785,...,38,0.043289,0.339958,38,0.030224,0.363197,38,0.019422,0.411375,37
42,pTau217,Avg_PCC,0.049406,-0.334664,35,0.193482,-0.231379,35,0.168273,-0.260667,...,35,0.342263,-0.171115,35,0.344978,-0.173292,35,0.932365,-0.015337,34
45,pTau217,Precun_gm,0.009976,-0.429772,35,0.035109,-0.365013,35,0.027505,-0.402863,...,35,0.116975,-0.276817,35,0.121544,-0.277985,35,0.520300,-0.114617,34
50,pTau231,MTL_gm,0.022692,0.389755,34,0.026657,0.420784,34,0.020614,0.475717,...,34,0.169592,0.252510,34,0.177368,0.252728,34,0.326453,0.203921,33


In [52]:
comp_para[(comp_para['composite_cor_p_baseline'] >= 0.05) & (comp_para['para_cor_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedi_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
19,Ab42_40,Temp_gm,0.08576,0.308553,32,0.147717,0.262477,32,0.158247,0.271622,...,32,0.042809,0.360078,32,0.047383,0.358819,32,0.062758,0.336959,31


In [53]:
(comp_para['composite_cor_p_baseline']  - comp_para['para_cor_p_baseline']).describe()

count    342.000000
mean      -0.005582
std        0.082067
min       -0.653308
25%        0.000000
50%        0.000000
75%        0.000000
max        0.444508
dtype: float64

## age

In [54]:
comp_para.loc[(comp_para['composite_mv_age_p_baseline'] < 0.05) & (comp_para['para_mv_age_p_baseline'] >= 0.05), ['var1', 'var2', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n', 'para_mv_age_p_baseline', 'para_mv_age_B_baseline',
       'para_mv_age_n']]

,var1,var2,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n
24,GFAP,Precun_gm,0.035642,-0.342781,38,0.109934,-0.266851,38
45,pTau217,Precun_gm,0.035109,-0.365013,35,0.116975,-0.276817,35
50,pTau231,MTL_gm,0.026657,0.420784,34,0.169592,0.252510,34


In [55]:
comp_para.columns

Index(['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline',
       'composite_cor_n', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n',
       'composite_mv_ageedu_p_baseline', 'composite_mv_ageedi_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type',
       'para_cor_p_baseline', 'para_cor_r_baseline', 'para_cor_n',
       'para_mv_age_p_baseline', 'para_mv_age_B_baseline', 'para_mv_age_n',
       'para_mv_ageedu_p_baseline', 'para_mv_ageedi_B_baseline',
       'para_mv_ageedu_n', 'para_mv_aeasr_p_baseline',
       'para_mv_aeasr_B_baseline', 'para_mv_aeasr_n'],
      dtype='object')

In [56]:
comp_para.loc[(comp_para['composite_mv_age_p_baseline'] >= 0.05) & (comp_para['para_mv_age_p_baseline'] < 0.05) ,['var1', 'var2','type', 'composite_mv_age_p_baseline', 'para_mv_age_p_baseline']]

,var1,var2,type,composite_mv_age_p_baseline,para_mv_age_p_baseline
14,Ab42_40,Avg_PCC,FDG PET,0.074935,0.017803
18,Ab42_40,Par_gm,FDG PET,0.150872,0.037223
19,Ab42_40,Temp_gm,FDG PET,0.147717,0.042809


In [57]:
(comp_para['composite_mv_age_p_baseline']  - comp_para['para_mv_age_p_baseline'])[(comp_para['composite_mv_age_p_baseline']  - comp_para['para_mv_age_p_baseline']) != 0].describe().round(2)

count    63.00
mean     -0.04
std       0.20
min      -0.41
25%      -0.19
50%      -0.04
75%       0.09
max       0.59
dtype: float64

In [58]:
(comp_para['composite_mv_age_p_baseline']  - comp_para['para_mv_age_p_baseline']).describe()

count    342.000000
mean      -0.007715
std        0.085958
min       -0.406856
25%        0.000000
50%        0.000000
75%        0.000000
max        0.591556
dtype: float64

In [59]:
(comp_para['composite_mv_age_n']  - comp_para['para_mv_age_n']).value_counts()

0    342
dtype: int64

## age education

In [60]:
comp_para[(comp_para['composite_mv_ageedu_p_baseline'] < 0.05) & (comp_para['para_mv_ageedu_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedi_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
24,GFAP,Precun_gm,0.152365,-0.236769,38,0.035642,-0.342781,38,0.046306,-0.345803,...,38,0.109934,-0.266851,38,0.119114,-0.263124,38,0.441246,-0.135554,37
45,pTau217,Precun_gm,0.009976,-0.429772,35,0.035109,-0.365013,35,0.027505,-0.402863,...,35,0.116975,-0.276817,35,0.121544,-0.277985,35,0.520300,-0.114617,34
50,pTau231,MTL_gm,0.022692,0.389755,34,0.026657,0.420784,34,0.020614,0.475717,...,34,0.169592,0.252510,34,0.177368,0.252728,34,0.326453,0.203921,33


In [61]:
comp_para[(comp_para['composite_mv_ageedu_p_baseline'] >= 0.05) & (comp_para['para_mv_ageedu_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedi_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
14,Ab42_40,Avg_PCC,0.062993,0.332478,32,0.074935,0.323079,32,0.077434,0.344107,...,32,0.017803,0.435210,32,0.020605,0.434633,32,0.009894,0.506670,31
18,Ab42_40,Par_gm,0.032843,0.378150,32,0.150872,0.263692,32,0.164547,0.262936,...,32,0.037223,0.388456,32,0.040092,0.389185,32,0.034491,0.420310,31
19,Ab42_40,Temp_gm,0.085760,0.308553,32,0.147717,0.262477,32,0.158247,0.271622,...,32,0.042809,0.360078,32,0.047383,0.358819,32,0.062758,0.336959,31


In [62]:
(comp_para['composite_mv_ageedu_p_baseline']  - comp_para['para_mv_ageedu_p_baseline']).describe()

count    342.000000
mean      -0.006537
std        0.079377
min       -0.402714
25%        0.000000
50%        0.000000
75%        0.000000
max        0.554942
dtype: float64

## age education apoe sex race

In [63]:
comp_para[(comp_para['composite_mv_aeasr_p_baseline'] < 0.05) & (comp_para['para_mv_aeasr_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedi_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
47,pTau217,Temp_gm,0.004333,-0.470576,35,0.005689,-0.462913,35,0.004235,-0.499990,...,35,0.039008,-0.355968,35,0.041863,-0.356632,35,0.065297,-0.309545,34
49,pTau231,Avg_PCC,0.918709,0.018182,34,0.292302,0.198722,34,0.279695,0.221808,...,34,0.489652,0.131588,34,0.499006,0.132471,34,0.172006,0.284367,33
55,pTau231,PostCing_gm,0.430767,0.139681,34,0.227325,0.216146,34,0.227280,0.226861,...,34,0.452331,0.135832,34,0.461404,0.135490,34,0.103545,0.319240,33


In [64]:
comp_para[(comp_para['composite_mv_aeasr_p_baseline'] >= 0.05) & (comp_para['para_mv_aeasr_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedi_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
18,Ab42_40,Par_gm,0.032843,0.37815,32,0.150872,0.263692,32,0.164547,0.262936,...,32,0.037223,0.388456,32,0.040092,0.389185,32,0.034491,0.42031,31


In [65]:
(comp_para['composite_mv_aeasr_p_baseline']  - comp_para['para_mv_aeasr_p_baseline']).describe()

count    342.000000
mean      -0.006909
std        0.090708
min       -0.483833
25%        0.000000
50%        0.000000
75%        0.000000
max        0.445145
dtype: float64

# Within Composite covariate effects

## no covariates vs age

In [66]:
results_composite.columns

Index(['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline',
       'composite_cor_n', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n',
       'composite_mv_ageedu_p_baseline', 'composite_mv_ageedi_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type'],
      dtype='object')

In [67]:
# significance lost by adding covariates
results_composite.loc[(results_composite['composite_cor_p_baseline']<0.05) & 
(results_composite['composite_mv_age_p_baseline'] >= 0.05),
['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline', 'composite_cor_n', 'composite_mv_age_p_baseline', 
'composite_mv_age_B_baseline', 'composite_mv_age_n', 'type']]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,type
14,Ab42,Avg_PCC,0.049509,0.350073,32,0.074420,0.325552,32,FDG PET
35,Ab42_40,Par_gm,0.032843,0.378150,32,0.150872,0.263692,32,FDG PET
76,pTau181,R_Hip,0.049985,0.320236,38,0.083752,0.302975,38,FDG PET
79,pTau217,Avg_PCC,0.049406,-0.334664,35,0.193482,-0.231379,35,FDG PET
102,pTau231,R_Hip,0.046601,0.343647,34,0.100807,0.307158,34,FDG PET
5,Ab40,pTau181,0.041350,-0.312467,43,0.117783,-0.259754,40,Plasma
13,Ab42,NFL,0.021309,0.367635,39,0.069592,0.305814,36,Plasma
16,Ab42,pTau231,0.041066,-0.337502,37,0.075697,-0.304300,34,Plasma
37,NFL,Ab42,0.021309,0.367635,39,0.069592,0.314980,36,Plasma
42,NFL,pTau217,0.023464,0.357652,40,0.324146,0.176876,38,Plasma


In [68]:
# significance gained by adding covariates
results_composite.loc[(results_composite['composite_cor_p_baseline']>=0.05) & 
(results_composite['composite_mv_age_p_baseline'] < 0.05),
['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline', 'composite_cor_n', 'composite_mv_age_p_baseline', 
'composite_mv_age_B_baseline', 'composite_mv_age_n', 'type']]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,type
31,Ab42_40,L_Hip,0.107911,0.289589,32,0.041741,0.381803,32,FDG PET
47,GFAP,Precun_gm,0.152365,-0.236769,38,0.035642,-0.342781,38,FDG PET
48,GFAP,Par_gm,0.073465,-0.293735,38,0.016666,-0.401410,38,FDG PET
70,pTau181,L_Hip,0.053497,0.315715,38,0.040522,0.351080,38,FDG PET
23,Ab42_40,pTau181,0.251058,0.188269,39,0.010760,0.420442,36,Plasma
25,Ab42_40,pTau231,0.156425,0.237790,37,0.004176,0.482884,34,Plasma
26,Ab42_40,pTau181_Ab42,0.210461,0.205061,39,0.022528,0.381548,36,Plasma
41,NFL,pTau181,0.088809,0.256630,45,0.040564,0.317781,42,Plasma
47,pTau181,Ab42_40,0.251058,0.188269,39,0.010760,0.431267,36,Plasma
49,pTau181,NFL,0.088809,0.256630,45,0.040564,0.324737,42,Plasma


In [69]:
# difference in p-values
(results_composite['composite_cor_p_baseline'] - results_composite['composite_mv_age_p_baseline'] ).describe()

count    396.000000
mean      -0.003304
std        0.212570
min       -0.665267
25%       -0.120835
50%       -0.002238
75%        0.096304
max        0.678102
dtype: float64

## age vs age and education

In [70]:
results_composite.columns

Index(['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline',
       'composite_cor_n', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n',
       'composite_mv_ageedu_p_baseline', 'composite_mv_ageedi_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type'],
      dtype='object')

In [71]:
# significance lost by adding covariates
results_composite.loc[(results_composite['composite_mv_age_p_baseline']<0.05) & 
(results_composite['composite_mv_ageedu_p_baseline'] >= 0.05),
['var1', 'var2', 'composite_mv_age_p_baseline',  'composite_mv_age_B_baseline', 'composite_mv_age_n',
       'composite_mv_ageedu_p_baseline', 'composite_mv_ageedi_B_baseline',  'composite_mv_ageedu_n', 'type']]

,var1,var2,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,composite_mv_ageedu_n,type
32,GFAP,pTau181,0.045342,0.314916,42,0.076072,0.295880,38,Plasma
48,pTau181,GFAP,0.045342,0.313735,42,0.076072,0.302963,38,Plasma
100,pTau181_Ab42,tma,0.047857,0.393763,30,0.058304,0.372999,30,Cognitive


In [89]:
# significance gained by adding covariates
results_composite.loc[(results_composite['composite_mv_age_p_baseline']>=0.05) & 
(results_composite['composite_mv_ageedu_p_baseline'] < 0.05),
['var1', 'var2', 'composite_mv_ageedi_B_baseline',  'composite_mv_ageedu_n', 'type']]

,var1,var2,composite_mv_ageedi_B_baseline,composite_mv_ageedu_n,type
76,pTau181,R_Hip,0.435799,38,FDG PET
43,NFL,pTau231,0.368909,33,Plasma
67,pTau231,NFL,0.363342,33,Plasma
18,Ab42,VOL_Inf_Mid_Fus_Temp_Lz,-0.546776,25,Volumetric
63,pTau181,VOL_Inf_Mid_Fus_Temp_Rz,0.384397,31,Volumetric
85,pTau231,VOL_Inf_Mid_Fus_Temp_Rz,0.441629,27,Volumetric
94,pTau181_Ab42,VOL_TotalGrayz,0.510871,25,Volumetric
95,pTau181_Ab42,VOL_Inf_Mid_Fus_Temp_Lz,0.612460,25,Volumetric
96,pTau181_Ab42,VOL_Inf_Mid_Fus_Temp_Rz,0.578515,25,Volumetric


In [73]:
# difference in p-values
(results_composite['composite_mv_age_p_baseline'] - results_composite['composite_mv_ageedu_p_baseline'] ).describe()

count    396.000000
mean       0.005631
std        0.097041
min       -0.305447
25%       -0.027536
50%       -0.000728
75%        0.035349
max        0.317561
dtype: float64

## age and education vs all 5 (age, education, apoe, sex, race)

In [74]:
# significance lost by adding covariates
results_composite.loc[(results_composite['composite_mv_ageedu_p_baseline']<0.05) & 
(results_composite['composite_mv_aeasr_p_baseline'] >= 0.05),
['var1', 'var2', 'composite_mv_ageedu_p_baseline', 'composite_mv_ageedi_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type']]

,var1,var2,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,composite_mv_ageedu_n,composite_mv_aeasr_p_baseline,composite_mv_aeasr_B_baseline,composite_mv_aeasr_n,type
29,Ab42_40,MTL_gm,0.013917,0.541882,32,0.095465,0.421478,31,FDG PET
31,Ab42_40,L_Hip,0.045449,0.393188,32,0.184307,0.298042,31,FDG PET
45,GFAP,FRONTAL_gm,0.041817,-0.353975,38,0.051642,-0.347176,37,FDG PET
47,GFAP,Precun_gm,0.046306,-0.345803,38,0.116234,-0.288459,37,FDG PET
48,GFAP,Par_gm,0.021884,-0.398042,38,0.071911,-0.318420,37,FDG PET
84,pTau217,FRONTAL_gm,0.033433,-0.390918,35,0.076097,-0.329712,34,FDG PET
86,pTau217,Precun_gm,0.027505,-0.402863,35,0.128133,-0.285928,34,FDG PET
94,pTau231,MTL_gm,0.020614,0.475717,34,0.071437,0.437641,33,FDG PET
33,GFAP,pTau217,0.004145,0.508438,35,0.115845,0.326815,34,Plasma
43,NFL,pTau231,0.042804,0.368909,33,0.069114,0.384116,32,Plasma


In [75]:
# significance gained by adding covariates
results_composite.loc[(results_composite['composite_mv_ageedu_p_baseline']>=0.05) & 
(results_composite['composite_mv_aeasr_p_baseline'] < 0.05),
['var1', 'var2', 'composite_mv_ageedu_p_baseline', 'composite_mv_ageedi_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type']]

,var1,var2,composite_mv_ageedu_p_baseline,composite_mv_ageedi_B_baseline,composite_mv_ageedu_n,composite_mv_aeasr_p_baseline,composite_mv_aeasr_B_baseline,composite_mv_aeasr_n,type
27,Ab42_40,Avg_PCC,0.077434,0.344107,32,0.017530,0.504921,31,FDG PET
77,pTau181,PostCing_gm,0.104171,0.277784,38,0.002779,0.530645,37,FDG PET
92,pTau231,Avg_PCC,0.279695,0.221808,34,0.048050,0.443415,33,FDG PET
103,pTau231,PostCing_gm,0.227280,0.226861,34,0.011087,0.524525,33,FDG PET
41,GFAP,tmb,0.084706,0.373753,20,0.000480,0.668144,20,Cognitive
78,pTau217,cowattotal,0.258085,-0.211466,35,0.035282,-0.382202,34,Cognitive
11,Ab42,VOL_Precun_Lz,0.080240,-0.389555,25,0.015601,-0.658255,24,Volumetric
14,Ab42,VOL_InfPar_Rz,0.088048,-0.390094,25,0.044304,-0.572154,24,Volumetric
17,Ab42,VOL_TotalGrayz,0.089914,-0.388724,25,0.036546,-0.642788,24,Volumetric
77,pTau231,VOL_Precun_Lz,0.185932,0.306308,27,0.033264,0.521732,26,Volumetric


In [76]:
# difference in p-values
(results_composite['composite_mv_ageedu_p_baseline'] - results_composite['composite_mv_aeasr_p_baseline'] ).describe()

count    396.000000
mean      -0.004607
std        0.203458
min       -0.757343
25%       -0.109428
50%       -0.006294
75%        0.086665
max        0.639226
dtype: float64

# Within New Para covariate effects

## no covariates vs age

In [77]:
# significance lost by adding covariates
results_new_para.loc[(results_new_para['new_para_cor_p_baseline']<0.05) & 
(results_new_para['new_para_mv_age_p_baseline'] >= 0.05),
['var1', 'var2', 'new_para_cor_p_baseline', 'new_para_cor_r_baseline', 'new_para_cor_n', 'new_para_mv_age_p_baseline', 
'new_para_mv_age_B_baseline', 'new_para_mv_age_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,type
0,Ab42,Precun_gm,0.043939,0.333116,37,0.067379,0.305760,36,FDG PET
1,Ab42_40,Par_gm,0.028343,0.360597,37,0.118173,0.262693,36,FDG PET
2,Ab42_40,Temp_gm,0.043318,0.334045,37,0.107181,0.266753,36,FDG PET
3,pTau217,SensMot_gm,0.042928,-0.325859,39,0.077152,-0.291165,38,FDG PET
4,pTau217,FRONTAL_gm,0.049762,-0.316344,39,0.050892,-0.317208,38,FDG PET
5,Ab40,pTau181,0.041350,-0.312467,43,0.117783,-0.259754,40,Plasma
6,Ab42,NFL,0.021309,0.367635,39,0.069592,0.305814,36,Plasma
7,Ab42,pTau231,0.041066,-0.337502,37,0.075697,-0.304300,34,Plasma
8,NFL,Ab42,0.021309,0.367635,39,0.069592,0.314980,36,Plasma
9,NFL,pTau217,0.023464,0.357652,40,0.324146,0.176876,38,Plasma


In [78]:
# significance gained by adding covariates
results_new_para.loc[(results_new_para['new_para_cor_p_baseline']>=0.05) & 
(results_new_para['new_para_mv_age_p_baseline'] < 0.05),
['var1', 'var2', 'new_para_cor_p_baseline', 'new_para_cor_r_baseline', 'new_para_cor_n', 'new_para_mv_age_p_baseline', 
'new_para_mv_age_B_baseline', 'new_para_mv_age_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,type
0,Ab42_40,Precun_gm,0.132015,0.252252,37,0.024590,0.372345,36,FDG PET
1,GFAP,Precun_gm,0.098998,-0.254918,43,0.018262,-0.372406,42,FDG PET
2,GFAP,Par_gm,0.085313,-0.265489,43,0.031328,-0.351754,42,FDG PET
3,Ab42_40,pTau181,0.251058,0.188269,39,0.010760,0.420442,36,Plasma
4,Ab42_40,pTau231,0.156425,0.237790,37,0.004176,0.482884,34,Plasma
5,Ab42_40,pTau181_Ab42,0.210461,0.205061,39,0.022528,0.381548,36,Plasma
6,NFL,pTau181,0.088809,0.256630,45,0.040564,0.317781,42,Plasma
7,pTau181,Ab42_40,0.251058,0.188269,39,0.010760,0.431267,36,Plasma
8,pTau181,NFL,0.088809,0.256630,45,0.040564,0.324737,42,Plasma
9,pTau231,Ab42_40,0.156425,0.237790,37,0.004176,0.488283,34,Plasma


In [79]:
# difference in p-values
(results_new_para['new_para_cor_p_baseline'] - results_new_para['new_para_mv_age_p_baseline'] ).describe()

count    360.000000
mean       0.000768
std        0.223001
min       -0.665267
25%       -0.123436
50%       -0.001886
75%        0.098388
max        0.701322
dtype: float64

## age vs age and education

In [80]:
# significance lost by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_age_p_baseline']<0.05) & 
(results_new_para['new_para_mv_ageedu_p_baseline'] >= 0.05),
['var1', 'var2', 'new_para_mv_age_p_baseline',  'new_para_mv_age_B_baseline', 'new_para_mv_age_n',
       'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedi_B_baseline',  'new_para_mv_ageedu_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,type
0,GFAP,RASref1_gm,0.027252,-0.365457,42,0.085702,-0.411186,38,FDG PET
1,GFAP,pTau181,0.045342,0.314916,42,0.076072,0.295880,38,Plasma
2,pTau181,GFAP,0.045342,0.313735,42,0.076072,0.302963,38,Plasma
3,pTau181_Ab42,tma,0.047857,0.393763,30,0.058304,0.372999,30,Cognitive


In [81]:
# significance gained by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_age_p_baseline']>=0.05) & 
(results_new_para['new_para_mv_ageedu_p_baseline'] < 0.05),
['var1', 'var2', 'new_para_mv_age_p_baseline',   'new_para_mv_age_B_baseline', 'new_para_mv_age_n',
       'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedi_B_baseline',  'new_para_mv_ageedu_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,type
0,Ab40,SensMot_gm,0.063042,0.301542,40,0.031438,0.400301,36,FDG PET
1,Ab42,SensMot_gm,0.091302,0.279539,36,0.044559,0.394320,32,FDG PET
2,pTau217,SensMot_gm,0.077152,-0.291165,38,0.021570,-0.440246,35,FDG PET
3,pTau217,FRONTAL_gm,0.050892,-0.317208,38,0.033321,-0.384078,35,FDG PET
4,NFL,pTau231,0.076470,0.297715,37,0.042804,0.368909,33,Plasma
5,pTau231,NFL,0.076470,0.300318,37,0.042804,0.363342,33,Plasma
6,Ab42,VOL_Inf_Mid_Fus_Temp_Lz,0.052459,-0.407944,25,0.017493,-0.546776,25,Volumetric
7,pTau181,VOL_Inf_Mid_Fus_Temp_Rz,0.262845,0.211083,31,0.045858,0.384397,31,Volumetric
8,pTau231,VOL_Inf_Mid_Fus_Temp_Rz,0.076130,0.358270,27,0.043423,0.441629,27,Volumetric
9,pTau181_Ab42,VOL_TotalGrayz,0.083519,0.372222,25,0.016493,0.510871,25,Volumetric


In [82]:
# difference in p-values
(results_new_para['new_para_mv_age_p_baseline'] - results_new_para['new_para_mv_ageedu_p_baseline'] ).describe()

count    360.000000
mean      -0.001020
std        0.121475
min       -0.637126
25%       -0.030787
50%       -0.000503
75%        0.036717
max        0.399605
dtype: float64

## age and education vs all 5 (age, education, apoe, sex, race)

In [83]:
# significance lost by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_ageedu_p_baseline']<0.05) & 
(results_new_para['new_para_mv_aeasr_p_baseline'] >= 0.05),
['var1', 'var2', 'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedi_B_baseline',
       'new_para_mv_ageedu_n', 'new_para_mv_aeasr_p_baseline',
       'new_para_mv_aeasr_B_baseline', 'new_para_mv_aeasr_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n,type
0,Ab42,SensMot_gm,0.044559,0.394320,32,0.082676,0.392566,31,FDG PET
1,Ab42_40,MTL_gm,0.013761,0.578730,32,0.094494,0.451093,31,FDG PET
2,GFAP,FRONTAL_gm,0.041756,-0.349772,38,0.051380,-0.343251,37,FDG PET
3,GFAP,Precun_gm,0.045915,-0.341150,38,0.115938,-0.284392,37,FDG PET
4,GFAP,Par_gm,0.021608,-0.379371,38,0.071454,-0.303397,37,FDG PET
5,pTau217,FRONTAL_gm,0.033321,-0.384078,35,0.075504,-0.324276,34,FDG PET
6,pTau217,Precun_gm,0.027162,-0.397276,35,0.127457,-0.281929,34,FDG PET
7,pTau231,MTL_gm,0.020698,0.504003,34,0.071912,0.463527,33,FDG PET
8,GFAP,pTau217,0.004145,0.508438,35,0.115845,0.326815,34,Plasma
9,NFL,pTau231,0.042804,0.368909,33,0.069114,0.384116,32,Plasma


In [84]:
# significance gained by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_ageedu_p_baseline']>=0.05) & 
(results_new_para['new_para_mv_aeasr_p_baseline'] < 0.05),
['var1', 'var2', 'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedi_B_baseline',
       'new_para_mv_ageedu_n', 'new_para_mv_aeasr_p_baseline',
       'new_para_mv_aeasr_B_baseline', 'new_para_mv_aeasr_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_mv_ageedu_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n,type
0,pTau181,PostCing_gm,0.104120,0.266144,38,0.002768,0.508466,37,FDG PET
1,pTau231,PostCing_gm,0.228012,0.224273,34,0.011149,0.518934,33,FDG PET
2,GFAP,tmb,0.084706,0.373753,20,0.000480,0.668144,20,Cognitive
3,pTau217,cowattotal,0.258085,-0.211466,35,0.035282,-0.382202,34,Cognitive
4,Ab42,VOL_Precun_Lz,0.080240,-0.389555,25,0.015601,-0.658255,24,Volumetric
5,Ab42,VOL_InfPar_Rz,0.088048,-0.390094,25,0.044304,-0.572154,24,Volumetric
6,Ab42,VOL_TotalGrayz,0.089914,-0.388724,25,0.036546,-0.642788,24,Volumetric
7,pTau231,VOL_Precun_Lz,0.185932,0.306308,27,0.033264,0.521732,26,Volumetric
8,pTau231,VOL_Precun_Rz,0.099630,0.385776,27,0.048666,0.493490,26,Volumetric
9,pTau181_Ab42,VOL_Precun_Lz,0.139112,0.320048,25,0.043465,0.489095,24,Volumetric


In [85]:
# difference in p-values
(results_new_para['new_para_mv_ageedu_p_baseline'] - results_new_para['new_para_mv_aeasr_p_baseline'] ).describe()

count    360.000000
mean      -0.001134
std        0.200943
min       -0.757343
25%       -0.098234
50%       -0.004973
75%        0.096365
max        0.639226
dtype: float64

## age vs all 5 (age, education, apoe, sex, race)

In [86]:
# significance lost by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_age_p_baseline']<0.05) & 
(results_new_para['new_para_mv_aeasr_p_baseline'] >= 0.05),
['var1', 'var2', 'new_para_mv_age_p_baseline', 'new_para_mv_ageedi_B_baseline',
       'new_para_mv_age_n', 'new_para_mv_aeasr_p_baseline',
       'new_para_mv_aeasr_B_baseline', 'new_para_mv_aeasr_n', 'type']].reset_index(drop= True)

,var1,var2,new_para_mv_age_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_age_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n,type
0,Ab42_40,MTL_gm,0.024846,0.578730,36,0.094494,0.451093,31,FDG PET
1,GFAP,FRONTAL_gm,0.022479,-0.349772,42,0.051380,-0.343251,37,FDG PET
2,GFAP,Precun_gm,0.018262,-0.341150,42,0.115938,-0.284392,37,FDG PET
3,GFAP,Par_gm,0.031328,-0.379371,42,0.071454,-0.303397,37,FDG PET
4,GFAP,RASref1_gm,0.027252,-0.411186,42,0.102489,-0.392753,37,FDG PET
5,pTau217,Precun_gm,0.028535,-0.397276,38,0.127457,-0.281929,34,FDG PET
6,pTau231,MTL_gm,0.042447,0.504003,38,0.071912,0.463527,33,FDG PET
7,GFAP,pTau181,0.045342,0.295880,42,0.266473,0.199430,37,Plasma
8,GFAP,pTau217,0.001443,0.508438,38,0.115845,0.326815,34,Plasma
9,pTau181,GFAP,0.045342,0.302963,42,0.266473,0.212568,37,Plasma


In [87]:
# significance gained by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_age_p_baseline']>=0.05) & 
(results_new_para['new_para_mv_aeasr_p_baseline'] < 0.05),
['var1', 'var2', 'new_para_mv_age_p_baseline', 'new_para_mv_ageedi_B_baseline',
       'new_para_mv_age_n', 'new_para_mv_aeasr_p_baseline',
       'new_para_mv_aeasr_B_baseline', 'new_para_mv_aeasr_n', 'type']].reset_index(drop= True)

,var1,var2,new_para_mv_age_p_baseline,new_para_mv_ageedi_B_baseline,new_para_mv_age_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n,type
0,Ab40,SensMot_gm,0.063042,0.400301,40,0.025934,0.440487,35,FDG PET
1,pTau181,PostCing_gm,0.250968,0.266144,42,0.002768,0.508466,37,FDG PET
2,pTau217,SensMot_gm,0.077152,-0.440246,38,0.024095,-0.422182,34,FDG PET
3,pTau231,PostCing_gm,0.260661,0.224273,38,0.011149,0.518934,33,FDG PET
4,GFAP,tmb,0.065837,0.373753,20,0.000480,0.668144,20,Cognitive
5,pTau217,cowattotal,0.276804,-0.211466,35,0.035282,-0.382202,34,Cognitive
6,Ab42,VOL_Precun_Lz,0.078980,-0.389555,25,0.015601,-0.658255,24,Volumetric
7,Ab42,VOL_InfPar_Rz,0.161693,-0.390094,25,0.044304,-0.572154,24,Volumetric
8,Ab42,VOL_TotalGrayz,0.137504,-0.388724,25,0.036546,-0.642788,24,Volumetric
9,Ab42,VOL_Inf_Mid_Fus_Temp_Lz,0.052459,-0.546776,25,0.009534,-0.778804,24,Volumetric


In [88]:
# difference in p-values
(results_new_para['new_para_mv_age_p_baseline'] - results_new_para['new_para_mv_aeasr_p_baseline'] ).describe()

count    360.000000
mean      -0.002154
std        0.236488
min       -0.709355
25%       -0.114805
50%       -0.005885
75%        0.116979
max        0.803247
dtype: float64